# monthly-builder

In [1]:
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup

# from funcs import create_game_info, create_team_info, create_info_df, create_boxscores, merge_boxscores, change_dtypes, create_PIE, commit_data

# import pandas as pd
# import numpy as np
# import string
# import time
# import sqlite3
# import warnings
# warnings.filterwarnings('ignore')

# pd.set_option('display.max_columns', None)

%run funcs.py

In [2]:
# global variables

# used to create id strings later
base_url = 'https://www.basketball-reference.com'

season_gamecount = 1

precovid_seasons = ['1314', '1415', '1516', '1617', '1718', '1819']
precovid_url_years = ['2014', '2015', '2016', '2017', '2018', '2019']
postcovid_seasons = ['1920', '2021', '2122', '2223']
postcovid_url_years = ['2020', '2021', '2022', '2023']

# post_covid_season_dict = {'1920': {'month_len': 8, 'final_month_gamecount': 83},
#                           '2021': {'month_len': 6, 'final_month_gamecount': 140},
#                           '2122': {'month_len': 7, 'final_month_gamecount': 83},
#                           '2223': {'month_len': 7, 'final_month_gamecount': 72}
#                          }

# used to create sql database table columns

info_columns = ['game_id', 'season', 'date', 'type', 'away_team', 'away_score', 'home_team', 'home_score', 'result']
num_columns = ['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
                'FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']

basic_stats = ['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']
# advanced_stats = ['FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']

rand = 6

In [ ]:
# connect to sql database
conn = sqlite3.connect('data/temp/monthly-builder.sqlite')
driver = webdriver.Chrome(service=svc,options=options)
time.sleep(rand)
game_type = 'regular'

season_id = '1314'
url_year = '2014'

In [ ]:
start_url = 'https://www.basketball-reference.com/leagues/NBA_' + url_year + '_games.html'

driver.get(start_url)
time.sleep(rand)

src = driver.page_source
# create beautiful soup object from html/xml
parser = BeautifulSoup(src, 'lxml')

# every month from the season
months = parser.find('div', attrs = {'class': 'filter'})
# partial urls for each month
links = months.findAll('a')
# full urls for each month
month_links = [base_url + link['href'] for link in links]

for month in month_links:

    driver.get(month)
    time.sleep(rand)
    month_src = driver.page_source
    month_parser = BeautifulSoup(month_src, 'lxml')
    month_table = month_parser.find('div', attrs = {'class': 'table_container is_setup'})
    
    # check if final month (apr). if true, set limit for game_urls before playoffs start
    final_reg_game_row = None
    splits = month_table.findAll('tr', attrs = {'class': 'thead'})
    for split in splits:
        if 'Playoffs' in split.text:
            final_reg_game_row = int(split['data-row'])
            break

    # get partial urls of every game in the month (if apr, stop before playoffs)
    if final_reg_game_row == None:

        game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
        game_urls = [base_url + url.a['href'] for url in game_partial_urls]

        for i in range(len(game_urls)):

            # driver.quit()
            # time.sleep(rand)
            # driver = webdriver.Chrome(service=svc,options=options)
            # time.sleep(rand)
            # driver.maximize_window()
            driver.get(game_urls[i])
            time.sleep(rand)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')

            # game_info database:

            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount,
                                         game_type=game_type)
            # will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]

            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # write game info to sql database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # team/player databases:

            # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # create team and player boxscores
            for i in range(len(stat_tables)):
                # split player and team boxscores
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

            # team_stats database:

            # combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # write to sql database
            team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

            # player_stats database:

            # combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # create team totals for PIE calculation
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            # add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # write to sql database
            player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)




            button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

            buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

            del buttons[6]
            del buttons[3]
            del buttons[0]

            game_length = len(buttons)

            button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            game_button_codes = button_codes[:game_length]

            for team_id in team_ids:
                for period in game_button_codes:

                    id_string = f'box-{team_id}-{period}-basic'
                    period_table = parser.find('table', attrs = {'id': id_string})

                    player_box, team_box = create_boxscores(period_table, game_id=game_id)

                    # team_stats database:

                    team_box.rename(columns={'Starters': 'team'}, inplace=True)
                    team_box['team'] = team_id
                    team_box.insert(loc=1, column='period', value=period)

                    team_box.reset_index(drop=True, inplace=True)
                    team_box = change_dtypes(team_box, basic_stats)

                    team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                    # player_stats database:

                    player_box.insert(loc=2, column='team', value=team_id)
                    player_box.insert(loc=1, column='period', value=period)

                    player_box.reset_index(drop=True, inplace=True)
                    player_box = change_dtypes(player_box, basic_stats)

                    player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1
            time.sleep(rand)

    elif final_reg_game_row != None:
        
        game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
        reg_game_urls = [base_url + url.a['href'] for url in game_partial_urls[:final_reg_game_row]]
        playoff_game_urls = [base_url + url.a['href'] for url in game_partial_urls[final_reg_game_row:]]

        for i in range(len(reg_game_urls)):

            # driver.quit()
            # time.sleep(rand)
            # driver = webdriver.Chrome(service=svc,options=options)
            # time.sleep(rand)
            # driver.maximize_window()
            driver.get(reg_game_urls[i])
            time.sleep(rand)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')

            # game_info database:

            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=reg_game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount,
                                         game_type=game_type)
            # will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]

            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # write game info to sql database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # team/player databases:

            # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # create team and player boxscores
            for i in range(len(stat_tables)):
                # split player and team boxscores
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

            # team_stats database:

            # combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # write to sql database
            team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

            # player_stats database:

            # combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # create team totals for PIE calculation
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            # add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # write to sql database
            player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)




            button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

            buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

            del buttons[6]
            del buttons[3]
            del buttons[0]

            game_length = len(buttons)

            button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            game_button_codes = button_codes[:game_length]

            for team_id in team_ids:
                for period in game_button_codes:

                    id_string = f'box-{team_id}-{period}-basic'
                    period_table = parser.find('table', attrs = {'id': id_string})

                    player_box, team_box = create_boxscores(period_table, game_id=game_id)

                    # team_stats database:

                    team_box.rename(columns={'Starters': 'team'}, inplace=True)
                    team_box['team'] = team_id
                    team_box.insert(loc=1, column='period', value=period)

                    team_box.reset_index(drop=True, inplace=True)
                    team_box = change_dtypes(team_box, basic_stats)

                    team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                    # player_stats database:

                    player_box.insert(loc=2, column='team', value=team_id)
                    player_box.insert(loc=1, column='period', value=period)

                    player_box.reset_index(drop=True, inplace=True)
                    player_box = change_dtypes(player_box, basic_stats)

                    player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1

        game_type = 'playoff'

        for i in range(len(playoff_game_urls)):

            # driver.quit()
            # time.sleep(rand)
            # driver = webdriver.Chrome(service=svc,options=options)
            # time.sleep(rand)
            driver.get(playoff_game_urls[i])
            time.sleep(rand)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')

            # game_info database:

            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=playoff_game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount,
                                         game_type=game_type)
            # will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]

            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # write game info to sql database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # team/player databases:

            # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # create team and player boxscores
            for i in range(len(stat_tables)):
                # split player and team boxscores
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

            # team_stats database:

            # combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # write to sql database
            team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

            # player_stats database:

            # combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # create team totals for PIE calculation
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            # add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # write to sql database
            player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)

            # quarterly boxscores:
            button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

            # button order: Game, Q1, Q2, H1, Q3, Q4, H2, (OT1, OT2, ...)
            buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

            del buttons[6]
            del buttons[3]
            del buttons[0]

            game_length = len(buttons)

            button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            game_button_codes = button_codes[:game_length]

            for team_id in team_ids:
                for period in game_button_codes:

                    id_string = f'box-{team_id}-{period}-basic'
                    period_table = parser.find('table', attrs = {'id': id_string})

                    player_box, team_box = create_boxscores(period_table, game_id=game_id)

                    # team_stats database:

                    team_box.rename(columns={'Starters': 'team'}, inplace=True)
                    team_box['team'] = team_id
                    team_box.insert(loc=1, column='period', value=period)

                    team_box = change_dtypes(team_box, basic_stats)

                    team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                    # player_stats database:

                    player_box.insert(loc=2, column='team', value=team_id)
                    player_box.insert(loc=1, column='period', value=period)

                    player_box = change_dtypes(player_box, basic_stats)

                    player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1

In [ ]:
# id_string = f'box-LAL-q1-basic'
# period_table = parser.find('table', attrs = {'id': id_string})

In [ ]:
query = 'SELECT * from team_stats_game'
game_info = pd.read_sql(query, con=conn)
# account for team name change following the 2013-2014 season
# game_info['away_team'].replace('CHA', 'CHO', inplace=True)
# game_info['home_team'].replace('CHA', 'CHO', inplace=True)
game_info

In [ ]:
game_info.info()

In [ ]:
game_info.loc[:,'FG':] = game_info.loc[:,'FG':].astype('float64')

In [ ]:
int.from_bytes(game_info['game_id'][0], byteorder='little')

In [ ]:
season_gamecount = 1

# connect to sql database
conn = sqlite3.connect('data/temp/1415-quarterly.sqlite')
driver = webdriver.Chrome(service=svc,options=options)
time.sleep(rand)
game_type = 'regular'

season_id = '1415'
url_year = '2015'

In [ ]:
start_url = 'https://www.basketball-reference.com/leagues/NBA_' + url_year + '_games.html'

driver.get(start_url)
time.sleep(rand)

src = driver.page_source
# create beautiful soup object from html/xml
parser = BeautifulSoup(src, 'lxml')

# every month from the season
months = parser.find('div', attrs = {'class': 'filter'})
# partial urls for each month
links = months.findAll('a')
# full urls for each month
month_links = [base_url + link['href'] for link in links]

for month in month_links:

    driver.get(month)
    time.sleep(rand)
    month_src = driver.page_source
    month_parser = BeautifulSoup(month_src, 'lxml')
    month_table = month_parser.find('div', attrs = {'class': 'table_container is_setup'})
    
    # check if final month (apr). if true, set limit for game_urls before playoffs start
    final_reg_game_row = None
    splits = month_table.findAll('tr', attrs = {'class': 'thead'})
    for split in splits:
        if 'Playoffs' in split.text:
            final_reg_game_row = int(split['data-row'])
            break

    # get partial urls of every game in the month (if apr, stop before playoffs)
    if final_reg_game_row == None:

        game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
        game_urls = [base_url + url.a['href'] for url in game_partial_urls]

        for i in range(len(game_urls)):

            # driver.quit()
            # time.sleep(rand)
            # driver = webdriver.Chrome(service=svc,options=options)
            # time.sleep(rand)
            # driver.maximize_window()
            driver.get(game_urls[i])
            time.sleep(rand)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')

            # game_info database:

            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount,
                                         game_type=game_type)
            # will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]

            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # write game info to sql database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # team/player databases:

            # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # create team and player boxscores
            for i in range(len(stat_tables)):
                # split player and team boxscores
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

            # team_stats database:

            # combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # write to sql database
            team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

            # player_stats database:

            # combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # create team totals for PIE calculation
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            # add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # write to sql database
            player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)




            button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

            buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

            del buttons[6]
            del buttons[3]
            del buttons[0]

            game_length = len(buttons)

            button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            game_button_codes = button_codes[:game_length]

            for team_id in team_ids:
                for period in game_button_codes:

                    id_string = f'box-{team_id}-{period}-basic'
                    period_table = parser.find('table', attrs = {'id': id_string})

                    player_box, team_box = create_boxscores(period_table, game_id=game_id)

                    # team_stats database:

                    team_box.rename(columns={'Starters': 'team'}, inplace=True)
                    team_box['team'] = team_id
                    team_box.insert(loc=1, column='period', value=period)

                    team_box.reset_index(drop=True, inplace=True)
                    team_box = change_dtypes(team_box, basic_stats)

                    team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                    # player_stats database:

                    player_box.insert(loc=2, column='team', value=team_id)
                    player_box.insert(loc=1, column='period', value=period)

                    player_box.reset_index(drop=True, inplace=True)
                    player_box = change_dtypes(player_box, basic_stats)

                    player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1
            time.sleep(rand)

    elif final_reg_game_row != None:
        
        game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
        reg_game_urls = [base_url + url.a['href'] for url in game_partial_urls[:final_reg_game_row]]
        playoff_game_urls = [base_url + url.a['href'] for url in game_partial_urls[final_reg_game_row:]]

        for i in range(len(reg_game_urls)):

            # driver.quit()
            # time.sleep(rand)
            # driver = webdriver.Chrome(service=svc,options=options)
            # time.sleep(rand)
            # driver.maximize_window()
            driver.get(reg_game_urls[i])
            time.sleep(rand)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')

            # game_info database:

            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=reg_game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount,
                                         game_type=game_type)
            # will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]

            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # write game info to sql database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # team/player databases:

            # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # create team and player boxscores
            for i in range(len(stat_tables)):
                # split player and team boxscores
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

            # team_stats database:

            # combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # write to sql database
            team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

            # player_stats database:

            # combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # create team totals for PIE calculation
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            # add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # write to sql database
            player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)




            button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

            buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

            del buttons[6]
            del buttons[3]
            del buttons[0]

            game_length = len(buttons)

            button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            game_button_codes = button_codes[:game_length]

            for team_id in team_ids:
                for period in game_button_codes:

                    id_string = f'box-{team_id}-{period}-basic'
                    period_table = parser.find('table', attrs = {'id': id_string})

                    player_box, team_box = create_boxscores(period_table, game_id=game_id)

                    # team_stats database:

                    team_box.rename(columns={'Starters': 'team'}, inplace=True)
                    team_box['team'] = team_id
                    team_box.insert(loc=1, column='period', value=period)

                    team_box.reset_index(drop=True, inplace=True)
                    team_box = change_dtypes(team_box, basic_stats)

                    team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                    # player_stats database:

                    player_box.insert(loc=2, column='team', value=team_id)
                    player_box.insert(loc=1, column='period', value=period)

                    player_box.reset_index(drop=True, inplace=True)
                    player_box = change_dtypes(player_box, basic_stats)

                    player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1

        game_type = 'playoff'

        for i in range(len(playoff_game_urls)):

            # driver.quit()
            # time.sleep(rand)
            # driver = webdriver.Chrome(service=svc,options=options)
            # time.sleep(rand)
            driver.get(playoff_game_urls[i])
            time.sleep(rand)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')

            # game_info database:

            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=playoff_game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount,
                                         game_type=game_type)
            # will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]

            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # write game info to sql database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # team/player databases:

            # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # create team and player boxscores
            for i in range(len(stat_tables)):
                # split player and team boxscores
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

            # team_stats database:

            # combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # write to sql database
            team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

            # player_stats database:

            # combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # create team totals for PIE calculation
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            # add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # write to sql database
            player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)

            # quarterly boxscores:
            button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

            # button order: Game, Q1, Q2, H1, Q3, Q4, H2, (OT1, OT2, ...)
            buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

            del buttons[6]
            del buttons[3]
            del buttons[0]

            game_length = len(buttons)

            button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            game_button_codes = button_codes[:game_length]

            for team_id in team_ids:
                for period in game_button_codes:

                    id_string = f'box-{team_id}-{period}-basic'
                    period_table = parser.find('table', attrs = {'id': id_string})

                    player_box, team_box = create_boxscores(period_table, game_id=game_id)

                    # team_stats database:

                    team_box.rename(columns={'Starters': 'team'}, inplace=True)
                    team_box['team'] = team_id
                    team_box.insert(loc=1, column='period', value=period)

                    team_box = change_dtypes(team_box, basic_stats)

                    team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                    # player_stats database:

                    player_box.insert(loc=2, column='team', value=team_id)
                    player_box.insert(loc=1, column='period', value=period)

                    player_box = change_dtypes(player_box, basic_stats)

                    player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1

In [ ]:
# driver.quit()
# conn.close()
# driver.quit()

In [ ]:
season_gamecount = 1

# connect to sql database
conn = sqlite3.connect('data/temp/1516-quarterly.sqlite')
driver = webdriver.Chrome(service=svc,options=options)
time.sleep(rand)
game_type = 'regular'

season_id = '1516'
url_year = '2016'

In [ ]:
start_url = 'https://www.basketball-reference.com/leagues/NBA_' + url_year + '_games.html'

driver.get(start_url)
time.sleep(rand)

src = driver.page_source
# create beautiful soup object from html/xml
parser = BeautifulSoup(src, 'lxml')

# every month from the season
months = parser.find('div', attrs = {'class': 'filter'})
# partial urls for each month
links = months.findAll('a')
# full urls for each month
month_links = [base_url + link['href'] for link in links]

for month in month_links:

    driver.get(month)
    time.sleep(rand)
    month_src = driver.page_source
    month_parser = BeautifulSoup(month_src, 'lxml')
    month_table = month_parser.find('div', attrs = {'class': 'table_container is_setup'})
    
    # check if final month (apr). if true, set limit for game_urls before playoffs start
    final_reg_game_row = None
    splits = month_table.findAll('tr', attrs = {'class': 'thead'})
    for split in splits:
        if 'Playoffs' in split.text:
            final_reg_game_row = int(split['data-row'])
            break

    # get partial urls of every game in the month (if apr, stop before playoffs)
    if final_reg_game_row == None:

        game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
        game_urls = [base_url + url.a['href'] for url in game_partial_urls]

        for i in range(len(game_urls)):

            # driver.quit()
            # time.sleep(rand)
            # driver = webdriver.Chrome(service=svc,options=options)
            # time.sleep(rand)
            # driver.maximize_window()
            driver.get(game_urls[i])
            time.sleep(rand)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')

            # game_info database:

            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount,
                                         game_type=game_type)
            # will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]

            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # write game info to sql database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # team/player databases:

            # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # create team and player boxscores
            for i in range(len(stat_tables)):
                # split player and team boxscores
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

            # team_stats database:

            # combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # write to sql database
            team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

            # player_stats database:

            # combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # create team totals for PIE calculation
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            # add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # write to sql database
            player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)




            button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

            buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

            del buttons[6]
            del buttons[3]
            del buttons[0]

            game_length = len(buttons)

            button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            game_button_codes = button_codes[:game_length]

            for team_id in team_ids:
                for period in game_button_codes:

                    id_string = f'box-{team_id}-{period}-basic'
                    period_table = parser.find('table', attrs = {'id': id_string})

                    player_box, team_box = create_boxscores(period_table, game_id=game_id)

                    # team_stats database:

                    team_box.rename(columns={'Starters': 'team'}, inplace=True)
                    team_box['team'] = team_id
                    team_box.insert(loc=1, column='period', value=period)

                    team_box.reset_index(drop=True, inplace=True)
                    team_box = change_dtypes(team_box, basic_stats)

                    team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                    # player_stats database:

                    player_box.insert(loc=2, column='team', value=team_id)
                    player_box.insert(loc=1, column='period', value=period)

                    player_box.reset_index(drop=True, inplace=True)
                    player_box = change_dtypes(player_box, basic_stats)

                    player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1
            time.sleep(rand)

    elif final_reg_game_row != None:
        
        game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
        reg_game_urls = [base_url + url.a['href'] for url in game_partial_urls[:final_reg_game_row]]
        playoff_game_urls = [base_url + url.a['href'] for url in game_partial_urls[final_reg_game_row:]]

        for i in range(len(reg_game_urls)):

            # driver.quit()
            # time.sleep(rand)
            # driver = webdriver.Chrome(service=svc,options=options)
            # time.sleep(rand)
            # driver.maximize_window()
            driver.get(reg_game_urls[i])
            time.sleep(rand)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')

            # game_info database:

            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=reg_game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount,
                                         game_type=game_type)
            # will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]

            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # write game info to sql database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # team/player databases:

            # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # create team and player boxscores
            for i in range(len(stat_tables)):
                # split player and team boxscores
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

            # team_stats database:

            # combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # write to sql database
            team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

            # player_stats database:

            # combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # create team totals for PIE calculation
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            # add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # write to sql database
            player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)




            button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

            buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

            del buttons[6]
            del buttons[3]
            del buttons[0]

            game_length = len(buttons)

            button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            game_button_codes = button_codes[:game_length]

            for team_id in team_ids:
                for period in game_button_codes:

                    id_string = f'box-{team_id}-{period}-basic'
                    period_table = parser.find('table', attrs = {'id': id_string})

                    player_box, team_box = create_boxscores(period_table, game_id=game_id)

                    # team_stats database:

                    team_box.rename(columns={'Starters': 'team'}, inplace=True)
                    team_box['team'] = team_id
                    team_box.insert(loc=1, column='period', value=period)

                    team_box.reset_index(drop=True, inplace=True)
                    team_box = change_dtypes(team_box, basic_stats)

                    team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                    # player_stats database:

                    player_box.insert(loc=2, column='team', value=team_id)
                    player_box.insert(loc=1, column='period', value=period)

                    player_box.reset_index(drop=True, inplace=True)
                    player_box = change_dtypes(player_box, basic_stats)

                    player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1

        game_type = 'playoff'

        for i in range(len(playoff_game_urls)):

            # driver.quit()
            # time.sleep(rand)
            # driver = webdriver.Chrome(service=svc,options=options)
            # time.sleep(rand)
            driver.get(playoff_game_urls[i])
            time.sleep(rand)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')

            # game_info database:

            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=playoff_game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount,
                                         game_type=game_type)
            # will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]

            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # write game info to sql database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # team/player databases:

            # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # create team and player boxscores
            for i in range(len(stat_tables)):
                # split player and team boxscores
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

            # team_stats database:

            # combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # write to sql database
            team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

            # player_stats database:

            # combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # create team totals for PIE calculation
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            # add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # write to sql database
            player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)

            # quarterly boxscores:
            button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

            # button order: Game, Q1, Q2, H1, Q3, Q4, H2, (OT1, OT2, ...)
            buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')

            del buttons[6]
            del buttons[3]
            del buttons[0]

            game_length = len(buttons)

            button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            game_button_codes = button_codes[:game_length]

            for team_id in team_ids:
                for period in game_button_codes:

                    id_string = f'box-{team_id}-{period}-basic'
                    period_table = parser.find('table', attrs = {'id': id_string})

                    player_box, team_box = create_boxscores(period_table, game_id=game_id)

                    # team_stats database:

                    team_box.rename(columns={'Starters': 'team'}, inplace=True)
                    team_box['team'] = team_id
                    team_box.insert(loc=1, column='period', value=period)

                    team_box = change_dtypes(team_box, basic_stats)

                    team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                    # player_stats database:

                    player_box.insert(loc=2, column='team', value=team_id)
                    player_box.insert(loc=1, column='period', value=period)

                    player_box = change_dtypes(player_box, basic_stats)

                    player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1

In [ ]:
driver.quit()
conn.close()

In [ ]:
# connect to sql database
conn = sqlite3.connect('data/temp/precovid-1617-1819-quarterly.sqlite')
driver = webdriver.Chrome(service=svc,options=options)
time.sleep(rand)

# season_id = '1617'
# url_year = '2017'

precovid_seasons = ['1617', '1718', '1819']
precovid_url_years = ['2017', '2018', '2019']
# postcovid_seasons = ['1920', '2021', '2122', '2223']
# postcovid_url_years = ['2020', '2021', '2022', '2023']

In [ ]:
for i in range(len(precovid_seasons)):

    season_gamecount = 1
    game_type = 'regular'
    season_id = precovid_seasons[i]
    url_year = precovid_url_years[i]
    
    start_url = 'https://www.basketball-reference.com/leagues/NBA_' + url_year + '_games.html'
    
    driver.get(start_url)
    time.sleep(rand)
    
    src = driver.page_source
    # create beautiful soup object from html/xml
    parser = BeautifulSoup(src, 'lxml')
    
    # every month from the season
    months = parser.find('div', attrs = {'class': 'filter'})
    # partial urls for each month
    links = months.findAll('a')
    # full urls for each month
    month_links = [base_url + link['href'] for link in links]
    
    for month in month_links:
    
        driver.get(month)
        time.sleep(rand)
        month_src = driver.page_source
        month_parser = BeautifulSoup(month_src, 'lxml')
        month_table = month_parser.find('div', attrs = {'class': 'table_container is_setup'})
        
        # check if final month (apr). if true, set limit for game_urls before playoffs start
        final_reg_game_row = None
        splits = month_table.findAll('tr', attrs = {'class': 'thead'})
        for split in splits:
            if 'Playoffs' in split.text:
                final_reg_game_row = int(split['data-row'])
                break
    
        # get partial urls of every game in the month (if apr, stop before playoffs)
        if final_reg_game_row == None:
    
            game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
            game_urls = [base_url + url.a['href'] for url in game_partial_urls]
    
            for i in range(len(game_urls)):
    
                # driver.quit()
                # time.sleep(rand)
                # driver = webdriver.Chrome(service=svc,options=options)
                # time.sleep(rand)
                # driver.maximize_window()
                driver.get(game_urls[i])
                time.sleep(rand)
                src = driver.page_source
                parser = BeautifulSoup(src, 'lxml')
    
                # game_info database:
    
                id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
                game_info = create_game_info(url=game_urls[i],
                                             season_id=season_id,
                                             season_gamecount=season_gamecount,
                                             game_type=game_type)
                # will use game_id with create_boxscores()
                game_id = game_info[0]
                team_info = create_team_info(id_table)
                # will use team_ids with merge_boxscores()
                team_ids = [team_info[0], team_info[2]]
    
                info_df = create_info_df(game_info=game_info,
                                         team_info=team_info,
                                         info_columns=info_columns)
                # write game info to sql database
                info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
    
                # team/player databases:
    
                # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
                stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
    
                player_box_list = [None, None, None, None]
                team_box_list = [None, None, None, None]
    
                # create team and player boxscores
                for i in range(len(stat_tables)):
                    # split player and team boxscores
                    player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
    
                # team_stats database:
    
                # combine boxscore and advanced boxscore for each team
                away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
                team_boxes = pd.concat([away_team_box, home_team_box])
                team_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                team_boxes = change_dtypes(team_boxes, num_columns)
                # write to sql database
                team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
    
                # player_stats database:
    
                # combine boxscore and advanced boxscore for each team
                away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
                player_boxes = pd.concat([away_player_box, home_player_box])
                player_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                player_boxes = change_dtypes(player_boxes, num_columns)
                # create team totals for PIE calculation
                totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
                # add PIE column to player boxscore
                player_boxes = create_PIE(player_boxes, totals)
                # write to sql database
                player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
    
    
    
    
                button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
    
                buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
    
                del buttons[6]
                del buttons[3]
                del buttons[0]
    
                game_length = len(buttons)
    
                button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
                game_button_codes = button_codes[:game_length]
    
                for team_id in team_ids:
                    for period in game_button_codes:
    
                        id_string = f'box-{team_id}-{period}-basic'
                        period_table = parser.find('table', attrs = {'id': id_string})
    
                        player_box, team_box = create_boxscores(period_table, game_id=game_id)
    
                        # team_stats database:
    
                        team_box.rename(columns={'Starters': 'team'}, inplace=True)
                        team_box['team'] = team_id
                        team_box.insert(loc=1, column='period', value=period)
    
                        team_box.reset_index(drop=True, inplace=True)
                        team_box = change_dtypes(team_box, basic_stats)
    
                        team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
    
                        # player_stats database:
    
                        player_box.insert(loc=2, column='team', value=team_id)
                        player_box.insert(loc=1, column='period', value=period)
    
                        player_box.reset_index(drop=True, inplace=True)
                        player_box = change_dtypes(player_box, basic_stats)
    
                        player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
    
                # increase gamecount to create next game_id
                season_gamecount += 1
                time.sleep(rand)
    
        elif final_reg_game_row != None:
            
            game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
            reg_game_urls = [base_url + url.a['href'] for url in game_partial_urls[:final_reg_game_row]]
            playoff_game_urls = [base_url + url.a['href'] for url in game_partial_urls[final_reg_game_row:]]
    
            for i in range(len(reg_game_urls)):
    
                # driver.quit()
                # time.sleep(rand)
                # driver = webdriver.Chrome(service=svc,options=options)
                # time.sleep(rand)
                # driver.maximize_window()
                driver.get(reg_game_urls[i])
                time.sleep(rand)
                src = driver.page_source
                parser = BeautifulSoup(src, 'lxml')
    
                # game_info database:
    
                id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
                game_info = create_game_info(url=reg_game_urls[i],
                                             season_id=season_id,
                                             season_gamecount=season_gamecount,
                                             game_type=game_type)
                # will use game_id with create_boxscores()
                game_id = game_info[0]
                team_info = create_team_info(id_table)
                # will use team_ids with merge_boxscores()
                team_ids = [team_info[0], team_info[2]]
    
                info_df = create_info_df(game_info=game_info,
                                         team_info=team_info,
                                         info_columns=info_columns)
                # write game info to sql database
                info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
    
                # team/player databases:
    
                # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
                stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
    
                player_box_list = [None, None, None, None]
                team_box_list = [None, None, None, None]
    
                # create team and player boxscores
                for i in range(len(stat_tables)):
                    # split player and team boxscores
                    player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
    
                # team_stats database:
    
                # combine boxscore and advanced boxscore for each team
                away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
                team_boxes = pd.concat([away_team_box, home_team_box])
                team_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                team_boxes = change_dtypes(team_boxes, num_columns)
                # write to sql database
                team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
    
                # player_stats database:
    
                # combine boxscore and advanced boxscore for each team
                away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
                player_boxes = pd.concat([away_player_box, home_player_box])
                player_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                player_boxes = change_dtypes(player_boxes, num_columns)
                # create team totals for PIE calculation
                totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
                # add PIE column to player boxscore
                player_boxes = create_PIE(player_boxes, totals)
                # write to sql database
                player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
    
    
    
    
                button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
    
                buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
    
                del buttons[6]
                del buttons[3]
                del buttons[0]
    
                game_length = len(buttons)
    
                button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
                game_button_codes = button_codes[:game_length]
    
                for team_id in team_ids:
                    for period in game_button_codes:
    
                        id_string = f'box-{team_id}-{period}-basic'
                        period_table = parser.find('table', attrs = {'id': id_string})
    
                        player_box, team_box = create_boxscores(period_table, game_id=game_id)
    
                        # team_stats database:
    
                        team_box.rename(columns={'Starters': 'team'}, inplace=True)
                        team_box['team'] = team_id
                        team_box.insert(loc=1, column='period', value=period)
    
                        team_box.reset_index(drop=True, inplace=True)
                        team_box = change_dtypes(team_box, basic_stats)
    
                        team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
    
                        # player_stats database:
    
                        player_box.insert(loc=2, column='team', value=team_id)
                        player_box.insert(loc=1, column='period', value=period)
    
                        player_box.reset_index(drop=True, inplace=True)
                        player_box = change_dtypes(player_box, basic_stats)
    
                        player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
    
                # increase gamecount to create next game_id
                season_gamecount += 1
    
            game_type = 'playoff'
    
            for i in range(len(playoff_game_urls)):
    
                # driver.quit()
                # time.sleep(rand)
                # driver = webdriver.Chrome(service=svc,options=options)
                # time.sleep(rand)
                driver.get(playoff_game_urls[i])
                time.sleep(rand)
                src = driver.page_source
                parser = BeautifulSoup(src, 'lxml')
    
                # game_info database:
    
                id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
                game_info = create_game_info(url=playoff_game_urls[i],
                                             season_id=season_id,
                                             season_gamecount=season_gamecount,
                                             game_type=game_type)
                # will use game_id with create_boxscores()
                game_id = game_info[0]
                team_info = create_team_info(id_table)
                # will use team_ids with merge_boxscores()
                team_ids = [team_info[0], team_info[2]]
    
                info_df = create_info_df(game_info=game_info,
                                         team_info=team_info,
                                         info_columns=info_columns)
                # write game info to sql database
                info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
    
                # team/player databases:
    
                # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
                stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
    
                player_box_list = [None, None, None, None]
                team_box_list = [None, None, None, None]
    
                # create team and player boxscores
                for i in range(len(stat_tables)):
                    # split player and team boxscores
                    player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
    
                # team_stats database:
    
                # combine boxscore and advanced boxscore for each team
                away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
                team_boxes = pd.concat([away_team_box, home_team_box])
                team_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                team_boxes = change_dtypes(team_boxes, num_columns)
                # write to sql database
                team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
    
                # player_stats database:
    
                # combine boxscore and advanced boxscore for each team
                away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
                player_boxes = pd.concat([away_player_box, home_player_box])
                player_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                player_boxes = change_dtypes(player_boxes, num_columns)
                # create team totals for PIE calculation
                totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
                # add PIE column to player boxscore
                player_boxes = create_PIE(player_boxes, totals)
                # write to sql database
                player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
    
                # quarterly boxscores:
                button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
    
                # button order: Game, Q1, Q2, H1, Q3, Q4, H2, (OT1, OT2, ...)
                buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
    
                del buttons[6]
                del buttons[3]
                del buttons[0]
    
                game_length = len(buttons)
    
                button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
                game_button_codes = button_codes[:game_length]
    
                for team_id in team_ids:
                    for period in game_button_codes:
    
                        id_string = f'box-{team_id}-{period}-basic'
                        period_table = parser.find('table', attrs = {'id': id_string})
    
                        player_box, team_box = create_boxscores(period_table, game_id=game_id)
    
                        # team_stats database:
    
                        team_box.rename(columns={'Starters': 'team'}, inplace=True)
                        team_box['team'] = team_id
                        team_box.insert(loc=1, column='period', value=period)
    
                        team_box = change_dtypes(team_box, basic_stats)
    
                        team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
    
                        # player_stats database:
    
                        player_box.insert(loc=2, column='team', value=team_id)
                        player_box.insert(loc=1, column='period', value=period)
    
                        player_box = change_dtypes(player_box, basic_stats)
    
                        player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
    
                # increase gamecount to create next game_id
                season_gamecount += 1

In [6]:
# connect to sql database
conn = sqlite3.connect('data/temp/postcovid-quarterly.sqlite')
driver = webdriver.Chrome(service=svc,options=options)
time.sleep(rand)

# season_id = '1617'
# url_year = '2017'

# precovid_seasons = ['1617', '1718', '1819']
# precovid_url_years = ['2017', '2018', '2019']
postcovid_seasons = ['2122', '2223']
postcovid_url_years = ['2022', '2023']

In [7]:
for i in range(len(postcovid_seasons)):
    
    season_id = postcovid_seasons[i]
    url_year = postcovid_url_years[i]
    season_gamecount = 1
    game_type = 'regular'
    start_url = 'https://www.basketball-reference.com/leagues/NBA_' + url_year + '_games.html'
    
    # open the season schedule page
    driver.get(start_url)
    time.sleep(rand)
    src = driver.page_source
    # create beautiful soup object from html/xml
    parser = BeautifulSoup(src, 'lxml')
    
    # every month from the season
    months = parser.find('div', attrs = {'class': 'filter'})
    # partial urls for each month
    links = months.findAll('a')
    # full urls for each month
    month_links = [base_url + link['href'] for link in links]

    
    # only include regular season months
    # month_links = month_links[0:post_covid_season_dict[season_id]['month_len']]

    
    for month_url in month_links:
        
        driver.get(month_url)
        time.sleep(rand)
        month_src = driver.page_source
        month_parser = BeautifulSoup(month_src, 'lxml')
        month_table = month_parser.find('div', attrs = {'class': 'table_container is_setup'})

        #check if there are play-in games this month
        play_in_str = month_table.findAll('td', string='Play-In Game')

        # if no play-in games:
        if not play_in_str:
        
            game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
            game_urls = [base_url + url.a['href'] for url in game_partial_urls]
        
            for i in range(len(game_urls)):
        
                # driver.quit()
                # time.sleep(rand)
                # driver = webdriver.Chrome(service=svc,options=options)
                # time.sleep(rand)
                # driver.maximize_window()
                driver.get(game_urls[i])
                time.sleep(rand)
                src = driver.page_source
                parser = BeautifulSoup(src, 'lxml')
        
                # game_info database:
        
                id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
                game_info = create_game_info(url=game_urls[i],
                                             season_id=season_id,
                                             season_gamecount=season_gamecount,
                                             game_type=game_type)
                # will use game_id with create_boxscores()
                game_id = game_info[0]
                team_info = create_team_info(id_table)
                # will use team_ids with merge_boxscores()
                team_ids = [team_info[0], team_info[2]]
        
                info_df = create_info_df(game_info=game_info,
                                         team_info=team_info,
                                         info_columns=info_columns)
                # write game info to sql database
                info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
        
                # team/player databases:
        
                # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
                stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
        
                player_box_list = [None, None, None, None]
                team_box_list = [None, None, None, None]
        
                # create team and player boxscores
                for i in range(len(stat_tables)):
                    # split player and team boxscores
                    player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
        
                # team_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
                team_boxes = pd.concat([away_team_box, home_team_box])
                team_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                team_boxes = change_dtypes(team_boxes, num_columns)
                # write to sql database
                team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
        
                # player_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
                player_boxes = pd.concat([away_player_box, home_player_box])
                player_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                player_boxes = change_dtypes(player_boxes, num_columns)
                # create team totals for PIE calculation
                totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
                # add PIE column to player boxscore
                player_boxes = create_PIE(player_boxes, totals)
                # write to sql database
                player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
                
                button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
        
                buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
                del buttons[6]
                del buttons[3]
                del buttons[0]
        
                game_length = len(buttons)
        
                button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
                game_button_codes = button_codes[:game_length]
        
                for team_id in team_ids:
                    for period in game_button_codes:
        
                        id_string = f'box-{team_id}-{period}-basic'
                        period_table = parser.find('table', attrs = {'id': id_string})
        
                        player_box, team_box = create_boxscores(period_table, game_id=game_id)
        
                        # team_stats database:
        
                        team_box.rename(columns={'Starters': 'team'}, inplace=True)
                        team_box['team'] = team_id
                        team_box.insert(loc=1, column='period', value=period)
        
                        team_box.reset_index(drop=True, inplace=True)
                        team_box = change_dtypes(team_box, basic_stats)
        
                        team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
        
                        # player_stats database:
        
                        player_box.insert(loc=2, column='team', value=team_id)
                        player_box.insert(loc=1, column='period', value=period)
        
                        player_box.reset_index(drop=True, inplace=True)
                        player_box = change_dtypes(player_box, basic_stats)
        
                        player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
        
                # increase gamecount to create next game_id
                season_gamecount += 1
                time.sleep(rand)
        
        elif play_in_str:
        
            play_in_rows = []
            for cell in play_in_str:
                parent = cell.find_parent()
                row = int(parent['data-row'])
                play_in_rows.append(row)
            
            first_play_in = play_in_rows[0]
            last_play_in = play_in_rows[-1]
            first_playoff = last_play_in + 1
            
            parent_table = play_in_str[0].find_parent().find_parent()
            slip_headers = parent_table.findAll('tr', attrs = {'class': 'thead'})
            
            for header in slip_headers:
                header_row = int(header['data-row'])
                if header_row < first_play_in:
                    first_play_in -= 1
                if header_row < last_play_in:
                    first_playoff -= 1
            
            game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
            reg_game_urls = [base_url + url.a['href'] for url in game_partial_urls[:first_play_in]]
            play_in_game_urls = [base_url + url.a['href'] for url in game_partial_urls[first_play_in:first_playoff]]
            playoff_game_urls = [base_url + url.a['href'] for url in game_partial_urls[first_playoff:]]
        
        
            for i in range(len(reg_game_urls)):
        
                # driver.quit()
                # time.sleep(rand)
                # driver = webdriver.Chrome(service=svc,options=options)
                # time.sleep(rand)
                # driver.maximize_window()
                driver.get(reg_game_urls[i])
                time.sleep(rand)
                src = driver.page_source
                parser = BeautifulSoup(src, 'lxml')
        
                # game_info database:
        
                id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
                game_info = create_game_info(url=reg_game_urls[i],
                                             season_id=season_id,
                                             season_gamecount=season_gamecount,
                                             game_type=game_type)
                # will use game_id with create_boxscores()
                game_id = game_info[0]
                team_info = create_team_info(id_table)
                # will use team_ids with merge_boxscores()
                team_ids = [team_info[0], team_info[2]]
        
                info_df = create_info_df(game_info=game_info,
                                         team_info=team_info,
                                         info_columns=info_columns)
                # write game info to sql database
                info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
        
                # team/player databases:
        
                # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
                stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
        
                player_box_list = [None, None, None, None]
                team_box_list = [None, None, None, None]
        
                # create team and player boxscores
                for i in range(len(stat_tables)):
                    # split player and team boxscores
                    player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
        
                # team_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
                team_boxes = pd.concat([away_team_box, home_team_box])
                team_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                team_boxes = change_dtypes(team_boxes, num_columns)
                # write to sql database
                team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
        
                # player_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
                player_boxes = pd.concat([away_player_box, home_player_box])
                player_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                player_boxes = change_dtypes(player_boxes, num_columns)
                # create team totals for PIE calculation
                totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
                # add PIE column to player boxscore
                player_boxes = create_PIE(player_boxes, totals)
                # write to sql database
                player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
                
                button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
        
                buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
                del buttons[6]
                del buttons[3]
                del buttons[0]
        
                game_length = len(buttons)
        
                button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
                game_button_codes = button_codes[:game_length]
        
                for team_id in team_ids:
                    for period in game_button_codes:
        
                        id_string = f'box-{team_id}-{period}-basic'
                        period_table = parser.find('table', attrs = {'id': id_string})
        
                        player_box, team_box = create_boxscores(period_table, game_id=game_id)
        
                        # team_stats database:
        
                        team_box.rename(columns={'Starters': 'team'}, inplace=True)
                        team_box['team'] = team_id
                        team_box.insert(loc=1, column='period', value=period)
        
                        team_box.reset_index(drop=True, inplace=True)
                        team_box = change_dtypes(team_box, basic_stats)
        
                        team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
        
                        # player_stats database:
        
                        player_box.insert(loc=2, column='team', value=team_id)
                        player_box.insert(loc=1, column='period', value=period)
        
                        player_box.reset_index(drop=True, inplace=True)
                        player_box = change_dtypes(player_box, basic_stats)
        
                        player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
        
                # increase gamecount to create next game_id
                season_gamecount += 1
        
            # game_type = 'play-in'
            # for i in range(len(play_in_game_urls)):
        
            #     # driver.quit()
            #     # time.sleep(rand)
            #     # driver = webdriver.Chrome(service=svc,options=options)
            #     # time.sleep(rand)
            #     # driver.maximize_window()
            #     driver.get(play_in_game_urls[i])
            #     time.sleep(rand)
            #     src = driver.page_source
            #     parser = BeautifulSoup(src, 'lxml')
        
            #     # game_info database:
        
            #     id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            #     game_info = create_game_info(url=play_in_game_urls[i],
            #                                  season_id=season_id,
            #                                  season_gamecount=season_gamecount,
            #                                  game_type=game_type)
            #     # will use game_id with create_boxscores()
            #     game_id = game_info[0]
            #     team_info = create_team_info(id_table)
            #     # will use team_ids with merge_boxscores()
            #     team_ids = [team_info[0], team_info[2]]
        
            #     info_df = create_info_df(game_info=game_info,
            #                              team_info=team_info,
            #                              info_columns=info_columns)
            #     # write game info to sql database
            #     info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
        
            #     # team/player databases:
        
            #     # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            #     stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
        
            #     player_box_list = [None, None, None, None]
            #     team_box_list = [None, None, None, None]
        
            #     # create team and player boxscores
            #     for i in range(len(stat_tables)):
            #         # split player and team boxscores
            #         player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
        
            #     # team_stats database:
        
            #     # combine boxscore and advanced boxscore for each team
            #     away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            #     team_boxes = pd.concat([away_team_box, home_team_box])
            #     team_boxes.reset_index(drop=True, inplace=True)
            #     # prepare numeric data
            #     team_boxes = change_dtypes(team_boxes, num_columns)
            #     # write to sql database
            #     team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
        
            #     # player_stats database:
        
            #     # combine boxscore and advanced boxscore for each team
            #     away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            #     player_boxes = pd.concat([away_player_box, home_player_box])
            #     player_boxes.reset_index(drop=True, inplace=True)
            #     # prepare numeric data
            #     player_boxes = change_dtypes(player_boxes, num_columns)
            #     # create team totals for PIE calculation
            #     totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            #     # add PIE column to player boxscore
            #     player_boxes = create_PIE(player_boxes, totals)
            #     # write to sql database
            #     player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
                
            #     button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
        
            #     buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
            #     del buttons[6]
            #     del buttons[3]
            #     del buttons[0]
        
            #     game_length = len(buttons)
        
            #     button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            #     game_button_codes = button_codes[:game_length]
        
            #     for team_id in team_ids:
            #         for period in game_button_codes:
        
            #             id_string = f'box-{team_id}-{period}-basic'
            #             period_table = parser.find('table', attrs = {'id': id_string})
        
            #             player_box, team_box = create_boxscores(period_table, game_id=game_id)
        
            #             # team_stats database:
        
            #             team_box.rename(columns={'Starters': 'team'}, inplace=True)
            #             team_box['team'] = team_id
            #             team_box.insert(loc=1, column='period', value=period)
        
            #             team_box.reset_index(drop=True, inplace=True)
            #             team_box = change_dtypes(team_box, basic_stats)
        
            #             team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
        
            #             # player_stats database:
        
            #             player_box.insert(loc=2, column='team', value=team_id)
            #             player_box.insert(loc=1, column='period', value=period)
        
            #             player_box.reset_index(drop=True, inplace=True)
            #             player_box = change_dtypes(player_box, basic_stats)
        
            #             player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
        
            #     # increase gamecount to create next game_id
            #     season_gamecount += 1
        
            
            game_type = 'playoff'
            for i in range(len(playoff_game_urls)):
        
                # driver.quit()
                # time.sleep(rand)
                # driver = webdriver.Chrome(service=svc,options=options)
                # time.sleep(rand)
                driver.get(playoff_game_urls[i])
                time.sleep(rand)
                src = driver.page_source
                parser = BeautifulSoup(src, 'lxml')
        
                # game_info database:
        
                id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
                game_info = create_game_info(url=playoff_game_urls[i],
                                             season_id=season_id,
                                             season_gamecount=season_gamecount,
                                             game_type=game_type)
                # will use game_id with create_boxscores()
                game_id = game_info[0]
                team_info = create_team_info(id_table)
                # will use team_ids with merge_boxscores()
                team_ids = [team_info[0], team_info[2]]
        
                info_df = create_info_df(game_info=game_info,
                                         team_info=team_info,
                                         info_columns=info_columns)
                # write game info to sql database
                info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
        
                # team/player databases:
        
                # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
                stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
        
                player_box_list = [None, None, None, None]
                team_box_list = [None, None, None, None]
        
                # create team and player boxscores
                for i in range(len(stat_tables)):
                    # split player and team boxscores
                    player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
        
                # team_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
                team_boxes = pd.concat([away_team_box, home_team_box])
                team_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                team_boxes = change_dtypes(team_boxes, num_columns)
                # write to sql database
                team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
        
                # player_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
                player_boxes = pd.concat([away_player_box, home_player_box])
                player_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                player_boxes = change_dtypes(player_boxes, num_columns)
                # create team totals for PIE calculation
                totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
                # add PIE column to player boxscore
                player_boxes = create_PIE(player_boxes, totals)
                # write to sql database
                player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
                # quarterly boxscores:
                button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
        
                # button order: Game, Q1, Q2, H1, Q3, Q4, H2, (OT1, OT2, ...)
                buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
                del buttons[6]
                del buttons[3]
                del buttons[0]
        
                game_length = len(buttons)
        
                button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
                game_button_codes = button_codes[:game_length]
        
                for team_id in team_ids:
                    for period in game_button_codes:
        
                        id_string = f'box-{team_id}-{period}-basic'
                        period_table = parser.find('table', attrs = {'id': id_string})
        
                        player_box, team_box = create_boxscores(period_table, game_id=game_id)
        
                        # team_stats database:
        
                        team_box.rename(columns={'Starters': 'team'}, inplace=True)
                        team_box['team'] = team_id
                        team_box.insert(loc=1, column='period', value=period)
        
                        team_box = change_dtypes(team_box, basic_stats)
        
                        team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
        
                        # player_stats database:
        
                        player_box.insert(loc=2, column='team', value=team_id)
                        player_box.insert(loc=1, column='period', value=period)
        
                        player_box = change_dtypes(player_box, basic_stats)
        
                        player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
        
                # increase gamecount to create next game_id
                season_gamecount += 1

In [ ]:
# driver.quit()
# conn.close()

In [ ]:
conn = sqlite3.connect('data/temp/postcovid-quarterly.sqlite')

In [8]:
query = 'SELECT * from game_info'
game_info = pd.read_sql(query, con=conn)
# # account for team name change following the 2013-2014 season
# # game_info['away_team'].replace('CHA', 'CHO', inplace=True)
# # game_info['home_team'].replace('CHA', 'CHO', inplace=True)
game_info

,game_id,season,date,type,away_team,away_score,home_team,home_score,result
0,192010220001,1920,2019-10-22,regular,NOP,122,TOR,130,1
1,192010220002,1920,2019-10-22,regular,LAL,102,LAC,112,1
2,192010230003,1920,2019-10-23,regular,CHI,125,CHO,126,1
3,192010230004,1920,2019-10-23,regular,DET,119,IND,110,0
4,192010230005,1920,2019-10-23,regular,CLE,85,ORL,94,1
...,...,...,...,...,...,...,...,...,...
4933,222306011310,2223,2023-06-01,playoff,MIA,93,DEN,104,1
4934,222306041311,2223,2023-06-04,playoff,MIA,111,DEN,108,0
4935,222306071312,2223,2023-06-07,playoff,DEN,109,MIA,94,0
4936,222306091313,2223,2023-06-09,playoff,DEN,108,MIA,95,0


In [ ]:
query = '''DELETE from game_info
WHERE game_id LIKE "2122%"
'''

cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [ ]:
query = 'SELECT * from team_stats_quarterly'
team_stats_quarterly = pd.read_sql(query, con=conn)
# # account for team name change following the 2013-2014 season
# # game_info['away_team'].replace('CHA', 'CHO', inplace=True)
# # game_info['home_team'].replace('CHA', 'CHO', inplace=True)
team_stats_quarterly

In [ ]:
gids = [int.from_bytes(x, byteorder='little') for x in team_stats_quarterly['game_id']]
gids

In [ ]:
team_stats_quarterly['game_id'] = gids

In [ ]:
team_stats_quarterly

In [ ]:
query = '''DELETE from team_stats_quarterly

'''

cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [ ]:
team_stats_quarterly.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

In [ ]:
query = '''DELETE from team_stats_quarterly
WHERE game_id LIKE "2122%"
'''

cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [ ]:
query = 'SELECT * from game_info'
team_stats_quarterly = pd.read_sql(query, con=conn)
# # account for team name change following the 2013-2014 season
# # game_info['away_team'].replace('CHA', 'CHO', inplace=True)
# # game_info['home_team'].replace('CHA', 'CHO', inplace=True)
team_stats_quarterly